In [120]:
# Modules
from scipy import ndimage
import numpy as np

import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras import optimizers

from matplotlib.pyplot import imshow

from PIL import Image, ImageFilter, ImageOps, ImageChops, ImageEnhance, ImageStat, ImageDraw
from __future__ import print_function
import glob
# from tf_unet import unet, util, image_util


from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

import sys
import datetime
import random
import warnings


# Functions
sys.path.insert(0, './helpers/')
from helpers import *

# Autoreload
%load_ext autoreload
%autoreload 2

# Set random seed
np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Ploting


In [200]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6

def prepare_standardplot(title, xlabel):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    ax1.set_ylabel('categorical cross entropy')
    ax1.set_xlabel(xlabel)
    ax1.set_yscale('log')
    ax2.set_ylabel('accuracy [% correct]')
    ax2.set_xlabel(xlabel)
    return fig, ax1, ax2

def finalize_standardplot(fig, ax1, ax2):
    ax1handles, ax1labels = ax1.get_legend_handles_labels()
    if len(ax1labels) > 0:
        ax1.legend(ax1handles, ax1labels)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels)
    fig.tight_layout()
    plt.subplots_adjust(top=0.9)

def plot_history(history, title):
    fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
    ax1.plot(history.history['loss'], label = "training")
    ax1.plot(history.history['val_loss'], label = "validation")
    ax2.plot(history.history['acc'], label = "training")
    ax2.plot(history.history['val_acc'], label = "validation")
    finalize_standardplot(fig, ax1, ax2)
    return fig


## Load the data

In [269]:
# Loads all data images in a list
filelist = glob.glob("../randomized_data/train/data_*.tiff")

# Sort the list according the their names 
# so that the the image and labels are on 
# the same position

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    return [ atoi(c) for c in re.split('(\d+)', text) ]

filelist.sort(key=natural_keys)

data =   [np.array(Image.open(fname)).flatten() for fname in filelist if fname[-9:-5] != 'mask']

labels = [np.array(Image.open(fname)).flatten() for fname in filelist if fname[-9:-5] == 'mask']

for lab in labels:
    
    lab[lab == 0] = 1
    lab[lab == 255] = 0
    
filelist = glob.glob("../randomized_data/test/data_*.tiff")

filelist.sort(key=natural_keys)

data_test =   [np.array(Image.open(fname)).flatten() for fname in filelist if fname[-9:-5] != 'mask']

labels_test = [np.array(Image.open(fname)).flatten() for fname in filelist if fname[-9:-5] == 'mask']

for lab in labels_test:
    
    lab[lab == 0] = 1
    lab[lab == 255] = 0

### simple model

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(480, 262144)
(1888, 262144)
(480, 262144)


In [287]:
print(data)

[[143. 144. 149. ... 120. 106. 130.]
 [ 94.  81. 142. ... 139. 136. 152.]
 [135. 114. 141. ... 150. 129. 168.]
 ...
 [105. 134. 105. ... 107. 103. 103.]
 [138. 115. 112. ... 143. 143. 126.]
 [125. 112. 140. ... 104. 132. 154.]]


In [294]:
def build_no_hidden_layers_model():
    model = Sequential() # set up empty model
    model.add(Dense(dimention, input_dim=dimention, activation='softmax', name = 'Y'))#input_shape = x_train.shape[1:]
    
    model.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    # model.summary()
    return model

In [295]:
model = build_no_hidden_layers_model()

history = model.fit(data, labels, batch_size=128, epochs= 100,  verbose=1,  validation_data = (data_test, labels_test))
plot_history(history,'titre')

Train on 1888 samples, validate on 480 samples
Epoch 1/100


MemoryError: 

In [ ]:
labels = np.array(labels, dtype=float)
data = np.array(data, dtype=float)
labels_test = np.array(labels_test, dtype=float)
data_test = np.array(data_test, dtype=float)
dimention = len(labels[1])
print(labels)
print(labels_test.shape)
print(data.shape)
print(data_test.shape)